In [19]:
# csv_inference_consumer.py

from kafka import KafkaConsumer, KafkaProducer
import os
import json
import uuid
from concurrent.futures import ThreadPoolExecutor


In [20]:

TOPIC_NAME = "avg-data"

KAFKA_SERVER = "kafka-server:29092"

NOTIFICATION_TOPIC = "NOTIFICATION"


In [21]:

consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=[KAFKA_SERVER],
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    # to deserialize kafka.producer.object into dict
    #value_deserializer=lambda m: json.loads(m.decode('utf-8')),
)


In [22]:
producer = KafkaProducer(
    bootstrap_servers = KAFKA_SERVER,
    api_version = (0, 11, 15)
)

In [23]:
def inferenceProcessFunction(data):
    # process steps with data in json format      
    
    json_data = json.loads(data)
    
    print("eventTime:" + str(json_data['eventTime']) + ' / ' + "nome:" + str(json_data['nome']) + ' / ' + "bpm:" + str(json_data['avgbpm']) + ' / ' + "temp:" + str(json_data['avgtemp']))
    
    if(json_data['avgbpm'] >= 120 or json_data['avgtemp'] >= 38):
        notification_data = data        
        producer.send(NOTIFICATION_TOPIC, notification_data)
        producer.flush()
        print('\nNotification Topic Sent!\n')
    

In [ ]:
for inf in consumer:
    print('processing line...')
    inf_data = inf.value        
    inferenceProcessFunction(inf_data)

processing line...
eventTime:2022-03-01 05:50:00 / nome:joao / bpm:42.0 / temp:34.0
processing line...
eventTime:2022-03-03 01:50:00 / nome:joao / bpm:111.0 / temp:35.0
processing line...
eventTime:2022-03-06 03:00:00 / nome:maria / bpm:69.0 / temp:34.0
processing line...
eventTime:2022-03-06 15:10:00 / nome:joao / bpm:64.0 / temp:36.0
processing line...
eventTime:2022-03-07 03:15:00 / nome:antonio / bpm:55.0 / temp:38.0

Notification Topic Sent!

processing line...
eventTime:2022-03-07 07:25:00 / nome:joao / bpm:70.0 / temp:38.0

Notification Topic Sent!

processing line...
eventTime:2022-03-09 03:30:00 / nome:maria / bpm:85.0 / temp:36.0
processing line...
eventTime:2022-03-13 01:35:00 / nome:maria / bpm:107.0 / temp:36.0
processing line...
eventTime:2022-03-14 23:30:00 / nome:maria / bpm:92.0 / temp:36.0
processing line...
eventTime:2022-03-17 13:40:00 / nome:antonio / bpm:60.0 / temp:37.0
processing line...
eventTime:2022-03-19 01:00:00 / nome:maria / bpm:83.0 / temp:37.0
processin